In [1]:
import pandas as pd
import keras 
import sklearn

Using TensorFlow backend.
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [2]:
from tqdm import tqdm
import spacy

In [3]:
df= pd.read_csv("acquisition1.csv")

In [20]:

df= df.drop_duplicates(subset='title',ignore_index=True)
df=df.dropna()

In [21]:
df.head()

,article_id,date,url,source,title,body
0,da0ecb17994c9d301fa938bcc047af56,14/04/20,https://www.environmental-expert.com/news/nede...,environmental-expert.com,Nederman completes acquisition of Environmenta...,Nederman Holding AB will take over as owner of...
1,228bf074db50b7980c6d372104eab972,14/04/20,https://www.tvnz.co.nz/one-news/new-zealand/pi...,tvnz.co.nz,Pilots help stock supermarket shelves as work ...,Stocking shelves at supermarkets for little mo...
2,66c352d987d4fab53740ed466906d082,14/04/20,https://www.digitalmusicnews.com/2020/04/13/ti...,digitalmusicnews.com,Ticketmaster Buys Company Founded by Ex-CEO Na...,"Ticketmaster has purchased Rival, a ticketing ..."
3,2683542352c6861cd9f25c8b0f829cbb,14/04/20,https://seekingalpha.com/article/4337405-xcel-...,seekingalpha.com,"Xcel Brands, Inc. (XELB) CEO Robert D'Loren on...","Andrew Berger - Vice President, SM Berger & Co..."
4,ba30e1e423d0dcd8e62f90c3f2c402c0,14/04/20,https://www.hstoday.us/federal-pages/departmen...,hstoday.us,"Department of Defense Contracts for April 13, ...","BFBC LLC, Bozeman, Montana, was awarded a $569..."


In [22]:
nlp=spacy.load('en_core_web_sm')
sentence_vectors = {}
docs=[]
for body in tqdm(df.body):
    document=nlp(str(body))
    docs.append(document)
    sentence_vectors.update({body:document.vector})
sentences=list(sentence_vectors.keys())
vectors=list(sentence_vectors.values())

100%|████████████████████████████████████████████████████████████████████████████████| 898/898 [04:55<00:00,  3.04it/s]


In [23]:
from sklearn.cluster import DBSCAN
import numpy as np
x= np.array(vectors)
n={} 
for i in tqdm(np.arange(0.001,1,0.002)):
    dbscan= DBSCAN(eps=i,min_samples=2,metric='cosine').fit(x)
    n.update({i:len(pd.Series(dbscan.labels_).value_counts())})
dbscan = DBSCAN(eps=0.08,min_samples=2,metric='cosine').fit(x)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:20<00:00, 23.82it/s]


In [27]:
print(pd.Series(dbscan.labels_))

0      0
1      0
2      0
3      0
4      0
      ..
884    0
885    0
886    0
887    0
888    0
Length: 889, dtype: int64


In [28]:
len(df)

898

In [ ]:
example_result=df[df.labels==2]

In [ ]:
example_result

In [ ]:
# knn
X=df[['vectors']].values
y=df[['labels']].values